In [ ]:
from dask.distributed import Client, progress

# client = Client("tcp://127.0.0.1:50328")
# client = Client(processes=False, threads_per_worker=3, n_workers=4, memory_limit="10GB")
# client = Client(n_workers=4)
client = Client()

client

In [ ]:
import pandas as pd

In [ ]:
parq = pd.read_parquet('local_table3.parquet.gzip')

In [ ]:
parq

In [ ]:
duplicateRowsPDF = parq[parq.duplicated(['personid'])]
duplicateRowsPDF.head(50)

In [ ]:
parq['personid']

In [ ]:
import dask
import dask.dataframe as dd

In [ ]:
df = dd.read_parquet('local_table3.parquet.gzip', engine='pyarrow')
df

In [ ]:
%time aa=df.groupby('personid')

In [ ]:
aa.count().compute()

In [ ]:
df.personid.compute()

Pandas'ta olan duplicate dask'ta yok

duplicateRowsDF = df[df.duplicated(['personid'])]

In [16]:
a=df.assign(temp_f=lambda x: x.personid)

In [17]:
a.head()

,personid,status,finger_index,cylindersbinary,temp_f
0,365538,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...,365538
1,365538,1,12,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...,365538
2,365538,1,13,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...,365538
3,365538,1,14,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...,365538
4,365538,1,15,b'\x00\x00\xff\x00\x00\x00\xff\x0f\x00\xf0\xff...,365538


In [130]:
sourceData = df[(df.personid == 365538) & (df.finger_index == 11)]
sourceData = sourceData['cylindersbinary'].compute().loc[0]
sourceData

b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xf0\xff\xff\x0f\x00\xff\xff\x00\x00/\x02\x000\xf0\x08\x02\x00\x00\x00\x06\x98\x99\x00\x00\x00\x99\x0f\x00\t\x00F\x00\x00\x04\x00\x00\x00\xf6\x00\x00\xb8\x00\x00\x00\xa3\x01\x00\x00Q\x01\x00\x00c\x00\x00\x00\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xf0\xff\xff\x0f\x00\xff\xff\x00\x00\x03\x00\x00\xc0\x92\t\x00\xdd\x00\x00d\xf0f\x00\x00 \x00\x00\x00`f\x00\x00\x00\x06@\x04\x00\x06@\x00\x08\x01\x00\x00\xa5\x00\x00\x00\xd6\x00\x00\x00b\x00\x00\x00\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xf0\xff\xff\x0f\x00\xff\xff\x00\x00\x0b`\x00`\x00\x00\x00&\x06`f\x00\x00\x00\x00\x06\x06\x00\x06\x02\x06`\x06\x00`b\x00\x00\x00b\x00\xe1\x00\x00\x00\x0e\x01\x00\x00O\x00\x00\x00b\x00\x00\x00\x00\xff\xff\x00\xf0\xff\xff\x00\xff\xff\xff\x0f\xff\xff\xff\x0f\xff\xff\xff\x0f\xff\xff\xff\x0f\xf0\xff\xff\x0f\x

In [124]:
import ctypes as ct
from ctypes import cdll

baseDir = '/home/dev/Practice/MCCLib-DASK'
ct.CDLL._func_restype_ = ct.c_float

libComp = cdll.LoadLibrary(baseDir + '/build/bin/fingercomp.so')
libComp

<CDLL '/home/dev/Practice/MCCLib-DASK/build/bin/fingercomp.so', handle 3df92a0 at 0x7f7387820dc0>

comparison_result = libComp.compareFingers(ct.c_char_p(sourceData),
                                           len(sourceData),
                                           ct.c_char_p(targetData),
                                           len(targetData))

In [126]:
# dnm source target ayni
comparison_result = libComp.compareFingers(ct.c_char_p(sourceData),
                                           len(sourceData),
                                           ct.c_char_p(sourceData),
                                           len(sourceData))

In [127]:
comparison_result

0.8684000372886658

newDF=df.assign(temp_f=lambda x: libComp.compareFingers(ct.c_char_p(sourceData),
                                           len(sourceData),
                                           ct.c_char_p(x.cylindersbinary),
                                           len(x.cylindersbinary)))

In [152]:
def myFunc(row):
#     print(row)
    print("type of row => ",type(row))
    print("type of row.cylindersbinary => ",type(row.cylindersbinary))
#     print("row.max() =>",row.max())
#     print("Bitti")
    
    print("row.cylindersbinary => ",len(row.cylindersbinary.compute()))

    return row.cylindersbinary

In [153]:
dnmdf=df.assign(temp_f=myFunc).compute()

type of row =>  <class 'dask.dataframe.core.DataFrame'>
type of row.cylindersbinary =>  <class 'dask.dataframe.core.Series'>
row.cylindersbinary =>  13150


In [141]:
dnmdf.head()

,personid,status,finger_index,cylindersbinary,temp_f
0,365538,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
1,365538,1,12,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
2,365538,1,13,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
3,365538,1,14,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
4,365538,1,15,b'\x00\x00\xff\x00\x00\x00\xff\x0f\x00\xf0\xff...,b'\x00\x00\xff\x00\x00\x00\xff\x0f\x00\xf0\xff...


In [82]:
type(a2['cylindersbinary'].compute())

pandas.core.series.Series

In [54]:
type(b.loc[117010].compute())

pandas.core.frame.DataFrame

In [104]:
sil = b.loc[365538]
sil = sil[sil.finger_index == 11]

sil = sil['cylindersbinary'].compute()
sil

personid
365538    b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
Name: cylindersbinary, dtype: object

In [112]:
type(sil)

pandas.core.series.Series

In [120]:
type(sil.loc[365538])

bytes

In [118]:
sil.loc[365538]

b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xf0\xff\xff\x0f\x00\xff\xff\x00\x00/\x02\x000\xf0\x08\x02\x00\x00\x00\x06\x98\x99\x00\x00\x00\x99\x0f\x00\t\x00F\x00\x00\x04\x00\x00\x00\xf6\x00\x00\xb8\x00\x00\x00\xa3\x01\x00\x00Q\x01\x00\x00c\x00\x00\x00\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xf0\xff\xff\x0f\x00\xff\xff\x00\x00\x03\x00\x00\xc0\x92\t\x00\xdd\x00\x00d\xf0f\x00\x00 \x00\x00\x00`f\x00\x00\x00\x06@\x04\x00\x06@\x00\x08\x01\x00\x00\xa5\x00\x00\x00\xd6\x00\x00\x00b\x00\x00\x00\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xf0\xff\xff\x0f\x00\xff\xff\x00\x00\x0b`\x00`\x00\x00\x00&\x06`f\x00\x00\x00\x00\x06\x06\x00\x06\x02\x06`\x06\x00`b\x00\x00\x00b\x00\xe1\x00\x00\x00\x0e\x01\x00\x00O\x00\x00\x00b\x00\x00\x00\x00\xff\xff\x00\xf0\xff\xff\x00\xff\xff\xff\x0f\xff\xff\xff\x0f\xff\xff\xff\x0f\xff\xff\xff\x0f\xf0\xff\xff\x0f\x

In [89]:
sil = b.loc[365538].compute()
sil

,status,finger_index,cylindersbinary
personid,,,
365538,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
365538,1,14,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
365538,1,12,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
365538,1,20,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
365538,1,19,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
365538,1,18,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
365538,1,17,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
365538,1,16,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
365538,1,15,b'\x00\x00\xff\x00\x00\x00\xff\x0f\x00\xf0\xff...


In [90]:
len(sil)

10

In [62]:
type(sil[:1])

pandas.core.frame.DataFrame

In [64]:
sil[:1].cylindersbinary

personid
117010    b'\x00\xff\xff\xff\xf0\xff\xff\xff\xff\xff\xff...
Name: cylindersbinary, dtype: object

In [39]:
b = df.set_index('personid')

sourceData = df.loc[365538].compute()
sourceData

KeyError: None

In [ ]:
a=df.assign(temp_f=lambda x: x.personid)

In [155]:
### Custom function with map partitionsa

In [172]:
import dask.dataframe as dd
df = pd.DataFrame({'x': [1, 2, 3, 4, 5, 1],
                   'y': [1., 2., 3., 4., 5., 1.]})
ddf = dd.from_pandas(df, npartitions=2)

In [173]:
ddf.compute()

,x,y
0,1,1.0
1,2,2.0
2,3,3.0
3,4,4.0
4,5,5.0
5,1,1.0


In [176]:
def myadd(row, a, b=1):
    print('Begin...')

    print('row.x =>', row.x)
    print('row.y =>', row.y)

    print('type(row.x) =>', type(row.x))
    print('type(row.y) =>', type(row.y))

    print('End...')

    return row.sum() + a + b

In [177]:
res = ddf.apply(myadd, axis=1, args=(2,), b=1.5)
# res = ddf.apply(myadd, axis=1, args=(2,), b=1.5, meta=('x', 'f8'))

Begin...
row.x => 1.0
row.y => 1.0
type(row.x) => <class 'numpy.float64'>
type(row.y) => <class 'numpy.float64'>
End...
Begin...
row.x => 1.0
row.y => 1.0
type(row.x) => <class 'numpy.float64'>
type(row.y) => <class 'numpy.float64'>
End...


In [178]:
res.compute()

0     5.5
1     7.5
2     9.5
3    11.5
4    13.5
5     5.5
dtype: float64